In [0]:
!cd cChess/ && git pull

In [0]:
!cd cChess && ls

In [0]:
!mkdir preview

In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# img = load_img('cChess/Labels/n/0.png')  # this is a PIL image
# x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=1,
#                           save_to_dir='preview', save_prefix='bishop', save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely

In [0]:
x.shape



In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50,50,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (1, 1)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(13))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [0]:



# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'cChess/Labels',  # this is the target directory
        target_size=(50, 50),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels


In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=3392 // batch_size,
        epochs=50)
model.save_weights('/cChess/Weights/first_try.h5')  # always save your weights after training or during training

In [0]:
!\cd cChess && mkdir Weights

In [0]:
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(
        'cChess/Labels/',  # this is the target directory
        target_size=(50, 50),  # all images will be resized to 150x150
        batch_size=10000,
        class_mode='categorical',
subset = 'training')  # since we use binary_crossentropy loss, we need binary labels
val_generator = train_datagen.flow_from_directory(
        'cChess/Labels/',  # this is the target directory
        target_size=(50, 50),  # all images will be resized to 150x150
        batch_size=10000,
        class_mode='categorical',
subset = 'validation')  # since we use binary_crossentropy loss, we need binary labels
x,y = train_generator.next()
x_test, y_test = val_generator.next()


In [0]:
import matplotlib.pyplot as plt
x.shape
plt.imshow(x[8])
print(y[8])

In [0]:
y.shape


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), input_shape=(50,50,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv2D(32, (1, 1)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())# this converts our 3D feature maps to 1D feature vectors
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(12))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x,y, batch_size = 64, validation_split = 0.1, epochs = 50)

In [0]:
y_pred = model.predict(x_test)
y_pred_rafael_otario = np.argmax(y_pred, axis = 1)
y_test_rafael_otario = np.argmax(y_test, axis = 1)
y_test_rafael_otario

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import numpy as np


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    print(unique_labels(y_true, y_pred))
  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)
class_names = ['B', 'K', 'N', 'P', 'Q', 'R', 'b', 'k', 'n', 'p', 'q', 'r']
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test_rafael_otario, y_pred_rafael_otario, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test_rafael_otario, y_pred_rafael_otario, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [0]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

epochs = 100
batch_size = 8


def save_bottlebeck_features(x,y, x_test, y_test):

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    bottleneck_features_train = model.predict(
        x)
    bottleneck_features_test = model.predict(
        x_test)
    print(bottleneck_features_train.shape)
    np.save(open('./cChess/Weights/bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)
    np.save(open('./cChess/Weights/bottleneck_labels.npy', 'wb'),
            y)

    np.save(open('./cChess/Weights/bottleneck_features_test.npy', 'wb'),
            bottleneck_features_test)
    np.save(open('./cChess/Weights/bottleneck_labels_test.npy', 'wb'),
            y_test)

def train_top_model():
    train_data = np.load(open('./cChess/Weights/bottleneck_features_train.npy', "rb"))
    train_labels = np.load(open('./cChess/Weights/bottleneck_labels.npy',"rb"))

    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(12, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_split=.2)
    model.save_weights("./cChess/Weights/bottleneck512.h5")

    return model
save_bottlebeck_features(x,y, x_test,y_test)
model = train_top_model()

In [0]:
# build the VGG16 network
bottleneck_features_test = np.load(open('./cChess/Weights/bottleneck_features_test.npy', "rb"))
y_test = np.load(open('./cChess/Weights/bottleneck_labels_test.npy',"rb"))
y_pred = model.predict(bottleneck_features_test)

plt.imshow(x_test[5])
print(y_pred[5])

In [0]:

y_pred_rafael_otario = np.argmax(y_pred, axis = 1)
y_test_rafael_otario = np.argmax(y_test, axis = 1)
y_test_rafael_otario

In [0]:
y_pred_rafael_otario

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import numpy as np


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    print(unique_labels(y_true, y_pred))
  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)
class_names = ['B', 'K', 'N', 'P', 'Q', 'R', 'b', 'k', 'n', 'p', 'q', 'r']
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test_rafael_otario, y_pred_rafael_otario, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test_rafael_otario, y_pred_rafael_otario, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [0]:
rafael_datagen = ImageDataGenerator(rescale=1./255)
rafael_generator = rafael_datagen.flow_from_directory(
        'cChess/RandomBoard_TestSets/Blender Test Sets/Test Labels',  # this is the target directory
        target_size=(50, 50),  # all images will be resized to 150x150
        batch_size=10000,
        class_mode='categorical')
x_rafael, y_rafael = rafael_generator.next()

In [0]:
rafael_bottleneck = model_vgg.predict(x_rafael)
rafael_pred = model.predict(rafael_bottleneck)

In [0]:
a= 9
plt.imshow(x_rafael[a])
print('Predict Probabilities: \n', rafael_pred[a])
print('Real Labels: \n' ,y_rafael[a])

In [0]:
from sklearn.metrics import confusion_matrix



In [0]:
import matplotlib.pyplot as plt

In [0]:
x.shape
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

In [0]:
!cd cChess/RandomBoard_TestSets/photo5021654756254394396/ && mkdir _

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'cChess/RandomBoard_TestSets/photo5021654756254394396/',  # this is the target directory
        target_size=(50, 50),  # all images will be resized to 150x150
        batch_size=64,
        class_mode='categorical')

In [0]:
x_test, y_test = test_generator.next()

In [0]:
bottleneck_features = model_vgg.predict(x_test)
model.evaluate(bottleneck_features,y_test)

In [0]:
i=15
plt.imshow(x_test[i])
print(np.argmax(y_test[i]))
bottleneck_features = model_vgg.predict(x_test[i].reshape((1,)+x_test[i].shape))
print(np.argmax(model.predict(bottleneck_features)))

In [0]:
i=1
plt.imshow(x[i])
print(np.argmax(y[i]))

bottleneck_features = model_vgg.predict(x[i].reshape((1,)+x[i].shape))
print(np.argmax(model.predict(bottleneck_features)))

In [0]:
a = np.array([
              [1,2,3],
              [4 ,5 ,6],
              [7,8,9],
              [10,11,12]
])

In [0]:
a

In [0]:
a.shape

In [0]:
(a == np.array([1,2,3])).T

In [0]:
y.shape

In [0]:
y_bin = np.amin((y==np.array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])), axis = 1)

In [0]:
i=18
plt.imshow(x[i])
print(y_bin[i])

In [0]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

epochs = 100
batch_size = 8


def save_bottlebeck_features_bin(x,y):

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    bottleneck_features_train = model.predict(
        x)
    print(bottleneck_features_train.shape)
    np.save(open('./cChess/Weights/bottleneck_features_train_bin.npy', 'wb'),
            bottleneck_features_train)
    np.save(open('./cChess/Weights/bottleneck_labels_bin.npy', 'wb'),
            y)

def train_top_model_bin():
    train_data = np.load(open('./cChess/Weights/bottleneck_features_train_bin.npy', "rb"))
    train_labels = np.load(open('./cChess/Weights/bottleneck_labels_bin.npy',"rb"))

    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_split=.2)
    model.save_weights("./cChess/Weights/bottleneck512_bin.h5")


save_bottlebeck_features_bin(x,y_bin)
train_top_model_bin()

In [0]:
!cd cChess && git status

In [0]:
!cd cChess && git add .

In [0]:
!cd cChess && git config --global user.email "afonso.delgado@hotmail.com"
!cd cChess && git config --global user.name "Afonso Delgado"


In [0]:
!cd cChess && git commit -m "trying some convnets architectures (94% accuracy on 13-classes classification and 99-100% accuracy on blank-piece classification on validation data)"

In [0]:
!cd cChess && git push